<a href="https://colab.research.google.com/github/xenophobed/Data-AI/blob/main/Finetune_PVS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Sep 18 07:45:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!git clone https://github.com/josuebatista/DL-Art-School.git
%cd DL-Art-School
!wget https://huggingface.co/Gatozu35/tortoise-tts/resolve/main/dvae.pth -O experiments/dvae.pth
!wget https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/autoregressive.pth -O experiments/autoregressive.pth
!pip install -r codes/requirements.laxed.txt

Cloning into 'DL-Art-School'...
remote: Enumerating objects: 16612, done.
remote: Total 16612 (delta 0), reused 0 (delta 0), pack-reused 16612 (from 1)
Receiving objects: 100% (16612/16612), 12.15 MiB | 20.00 MiB/s, done.
Resolving deltas: 100% (13325/13325), done.
/content/DL-Art-School
--2024-09-18 07:21:57--  https://huggingface.co/Gatozu35/tortoise-tts/resolve/main/dvae.pth
Resolving huggingface.co (huggingface.co)... 13.35.166.114, 13.35.166.69, 13.35.166.50, ...
Connecting to huggingface.co (huggingface.co)|13.35.166.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/9f/83/9f8300e5ccd418d283e72c07d1851f269dbd2ca7bae49cee21285965d9bebe14/a990825371506c16bcf0e8167bf24ccf82f65bb6a1dbcbfcf058d76f9b197e35?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27dvae.pth%3B+filename%3D%22dvae.pth%22%3B&Expires=1726903318&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNj

In [ ]:
'''
Downgrading Transformers to previous version.
It seems that last release of transformers broke the model loader.
'''
!pip install transformers==4.29.2

# Check the integrity of the dVAE checkpoint
!sha256sum /content/DL-Art-School/experiments/dvae.pth | grep a990825371506c16bcf0e8167bf24ccf82f65bb6a1dbcbfcf058d76f9b197e35 || echo "SOMETHING IS WRONG WITH THE CHECKPOINT; REPORT THIS AS A GITHUB ISSUE AND DO NOT PROCEED"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
a990825371506c16bcf0e8167bf24ccf82f65bb6a1dbcbfcf058d76f9b197e35  /content/DL-Art-School/experiments/dvae.pth


In [ ]:
from pathlib import Path
from math import ceil
DEFAULT_TRAIN_BS = 64
DEFAULT_VAL_BS = 32
#@markdown # Hyperparameter calculation
#@markdown Run this cell to obtain suggested parameters for training
Dataset_Training_Path = "/content/gdrive/MyDrive/output/Learn_OAI_Whisper_Sample_Audio01.mp3_2024_09_18-07_43/train.txt" #@param {type:"string"}
ValidationDataset_Training_Path = "/content/gdrive/MyDrive/output/Learn_OAI_Whisper_Sample_Audio01.mp3_2024_09_18-07_43/valid.txt" #@param {type:"string"}

#@markdown ### **NOTE**: Dataset must be in the following format.

#@markdown  `dataset/`
#@markdown * ---├── `val.txt`
#@markdown * ---├── `train.txt`
#@markdown * ---├── `wavs/`

#@markdown `wavs/` directory must contain `.wav` files.

#@markdown  Example for `train.txt` and `val.txt`:

#@markdown * `wavs/A.wav|Write the transcribed audio here.`

#@markdown todo: actually check the dataset structure

if Dataset_Training_Path == ValidationDataset_Training_Path:
  print("WARNING: training dataset path == validation dataset path!!!")
  print("\tThis is technically okay but will make all of the validation metrics useless. ")
  print("it will also SUBSTANTIALLY slow down the rate of training, because validation datasets are supposed to be much smaller than training ones.")

def txt_file_lines(p: str) -> int:
  return len(Path(p).read_text().strip().split('\n'))
training_samples = txt_file_lines(Dataset_Training_Path)
val_samples = txt_file_lines(ValidationDataset_Training_Path)

if training_samples < 128: print("WARNING: very small dataset! the smallest dataset tested thus far had ~200 samples.")
if val_samples < 20: print("WARNING: very small validation dataset! val batch size will be scaled down to account")

def div_spillover(n: int, bs: int) -> int: # returns new batch size
  epoch_steps,remain = divmod(n,bs)
  if epoch_steps*2 > bs: return bs # don't bother optimising this stuff if epoch_steps are high
  if not remain: return bs # unlikely but still

  if remain*2 < bs: # "easier" to get rid of remainder -- should increase bs
    target_bs = n//epoch_steps
  else: # easier to increase epoch_steps by 1 -- decrease bs
    target_bs = n//(epoch_steps+1)
  assert n%target_bs < epoch_steps+2 # should be very few extra
  return target_bs

if training_samples < DEFAULT_TRAIN_BS:
  print("WARNING: dataset is smaller than a single batch. This will almost certainly perform poorly. Trying anyway")
  train_bs = training_samples
else:
  train_bs = div_spillover(training_samples, DEFAULT_TRAIN_BS)
if val_samples < DEFAULT_VAL_BS:
  val_bs = val_samples
else:
  val_bs = div_spillover(val_samples, DEFAULT_VAL_BS)

steps_per_epoch = training_samples//train_bs
lr_decay_epochs = [20, 40, 56, 72]
lr_decay_steps = [steps_per_epoch * e for e in lr_decay_epochs]
print_freq = min(100, max(20, steps_per_epoch))
val_freq = save_checkpoint_freq = print_freq * 3

print("===CALCULATED SETTINGS===")
print(f'{train_bs=} {val_bs=}')
print(f'{val_freq=} {lr_decay_steps=}')
print(f'{print_freq=} {save_checkpoint_freq=}')

===CALCULATED SETTINGS===
train_bs=4 val_bs=1
val_freq=60 lr_decay_steps=[20, 40, 56, 72]
print_freq=20 save_checkpoint_freq=60


In [ ]:
#@markdown ##_Settings for normal users:_
Experiment_Name = "Learn_OAI_Whisper_20240411_JRB" #@param {type:"string"}
Dataset_Training_Name= "TestDataset" #@param {type:"string"}
ValidationDataset_Name = "TestValidation" # this seems to be useless??? @param {type:"string"}
SaveTrainingStates = False # @param {type:"boolean"}
Keep_Last_N_Checkpoints = 0 #@param {type:"slider", min:0, max:10, step:1}
#@markdown * **NOTE**: 0 means "keep all models saved", which could potentially cause out-of-storage issues.
#@markdown * Without training states, each model "only" takes up ~1.6GB. You should have ~50GB of free space to begin with.
#@markdown * With training states, each model (pth+state) takes up ~4.9 GB; Colab will crash around ~10 undeleted checkpoints in this case.

#@markdown ##_Other training parameters_
Fp16 = False #@param {type:"boolean"}
Use8bit = True #@param {type:"boolean"}
#@markdown * **NOTE**: for some reason, fp16 does not seem to improve vram use when combined with 8bit [citation needed]. To be verified later...
TrainingRate = "1e-5" #@param {type:"string"}
TortoiseCompat = False #@param {type:"boolean"}

#@markdown * **NOTE**: TortoiseCompat introduces some breaking changes to the training process. **If you want to reproduce older models**, disable this checkbox.

#@markdown ##_Calculated settings_ override
#@markdown #####Blank entries rely on the calculated defaults from the cell above.
#@markdown ######**Leave them blank unless you want to adjust them manually**
TrainBS = "" #@param {type:"string"}
ValBS = "" #@param {type:"string"}
ValFreq = "" #@param {type:"string"}
LRDecaySteps = "" #@param {type:"string"}
PrintFreq = "" #@param {type:"string"}
SaveCheckpointFreq = "" #@param {type:"string"}

def take(orig, override):
  if override == "": return orig
  return type(orig)(override)

train_bs = take(train_bs, TrainBS)
val_bs = take(val_bs, ValBS)
val_freq = take(val_freq, ValFreq)
lr_decay_steps = eval(LRDecaySteps) if LRDecaySteps else lr_decay_steps
print_freq = take(print_freq, PrintFreq)
save_checkpoint_freq = take(save_checkpoint_freq, SaveCheckpointFreq)
assert len(lr_decay_steps) == 4
gen_lr_steps = ', '.join(str(v) for v in lr_decay_steps)

#@markdown #Run this cell after you finish editing the settings.

In [ ]:

%cd /content/DL-Art-School
# !wget https://raw.githubusercontent.com/152334H/DL-Art-School/master/experiments/EXAMPLE_gpt.yml -O experiments/EXAMPLE_gpt.yml
!wget https://raw.githubusercontent.com/josuebatista/DL-Art-School/master/experiments/EXAMPLE_gpt.yml -O experiments/EXAMPLE_gpt.yml

import os
%cd /content/DL-Art-School
!sed -i 's/batch_size: 128/batch_size: '"$train_bs"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/batch_size: 64/batch_size: '"$val_bs"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/val_freq: 500/val_freq: '"$val_freq"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/500, 1000, 1400, 1800/'"$gen_lr_steps"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/print_freq: 100/print_freq: '"$print_freq"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/save_checkpoint_freq: 500/save_checkpoint_freq: '"$save_checkpoint_freq"'/g' ./experiments/EXAMPLE_gpt.yml

!sed -i 's+CHANGEME_validation_dataset_name+'"$ValidationDataset_Name"'+g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's+CHANGEME_path_to_validation_dataset+'"$ValidationDataset_Training_Path"'+g' ./experiments/EXAMPLE_gpt.yml
if(Fp16==True):
  os.system("sed -i 's+fp16: false+fp16: true+g' ./experiments/EXAMPLE_gpt.yml")
!sed -i 's/use_8bit: true/use_8bit: '"$Use8bit"'/g' ./experiments/EXAMPLE_gpt.yml

!sed -i 's/disable_state_saving: true/disable_state_saving: '"$SaveTrainingStates"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/tortoise_compat: True/tortoise_compat: '"$TortoiseCompat"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/number_of_checkpoints_to_save: 0/number_of_checkpoints_to_save: '"$Keep_Last_N_Checkpoints"'/g' ./experiments/EXAMPLE_gpt.yml


!sed -i 's/CHANGEME_training_dataset_name/'"$Dataset_Training_Name"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/CHANGEME_your_experiment_name/'"$Experiment_Name"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's+CHANGEME_path_to_training_dataset+'"$Dataset_Training_Path"'+g' ./experiments/EXAMPLE_gpt.yml


if (not TrainingRate=="1e-5"):
  os.system("sed -i 's+!!float 1e-5 # CHANGEME:+!!float '" + TrainingRate + "' #+g' ./experiments/EXAMPLE_gpt.yml")

# Print the contents of the file
with open('./experiments/EXAMPLE_gpt.yml', 'r') as configfile:
    print(configfile.read())

/content/DL-Art-School
--2024-09-18 07:46:50--  https://raw.githubusercontent.com/josuebatista/DL-Art-School/master/experiments/EXAMPLE_gpt.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6489 (6.3K) [text/plain]
Saving to: ‘experiments/EXAMPLE_gpt.yml’

experiments/EXAMPLE 100%[===================>]   6.34K  --.-KB/s    in 0s      

2024-09-18 07:46:50 (87.7 MB/s) - ‘experiments/EXAMPLE_gpt.yml’ saved [6489/6489]

/content/DL-Art-School
name: Learn_OAI_Whisper_20240411_JRB
model: extensibletrainer
scale: 1
gpu_ids: [0] # <-- unless you have multiple gpus, use this
start_step: 0 # -1 causes 0.pth to be saved!
checkpointing_enabled: true  # <-- Gradient checkpointing. Enable for huge GPU memory savings. Disable for distributed training.
fp16: false # TODO

In [ ]:
%cd /content/DL-Art-School/codes

!python3 train.py -opt ../experiments/EXAMPLE_gpt.yml

/content/DL-Art-School/codes
/content/DL-Art-School/codes/trainer/base_model.py:5: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
Disabled distributed training.
Path already exists. Rename it to [/content/DL-Art-School/experiments/Learn_OAI_Whisper_20240411_JRB_archived_240918-074703]
24-09-18 07:47:03.565 - INFO:   name: Learn_OAI_Whisper_20240411_JRB
  model: extensibletrainer
  scale: 1
  gpu_ids: [0]
  start_step: 0
  checkpointing_enabled: True
  fp16: False
  use_8bit: True
  wandb: False
  use_tb_logger: True
  datasets:[
    train:[
      name: TestDataset
      n_workers: 8
      batch_size: 4
      mode: paired_voice_audio
      path: /content/gdrive/MyDrive/output/Learn_OAI_Whisper_Sample_Audio01.mp3_2024_09_18-07_43/train.txt
      fetcher_mode: ['lj']
      phase: train
      

In [ ]:
!cp -r /content/DL-Art-School/experiments/$Experiment_Name /content/gdrive/MyDrive/

In [ ]:
!git clone https://github.com/152334H/tortoise-tts-fast
%cd tortoise-tts-fast
!pip3 install -r requirements.txt --no-deps
!pip3 install -e .

Cloning into 'tortoise-tts-fast'...
remote: Enumerating objects: 2314, done.
remote: Total 2314 (delta 0), reused 0 (delta 0), pack-reused 2314 (from 1)
Receiving objects: 100% (2314/2314), 105.07 MiB | 23.64 MiB/s, done.
Resolving deltas: 100% (1070/1070), done.
/content/tortoise-tts-fast
Ignoring backports-zoneinfo: markers 'python_version >= "3.8" and python_version < "3.9"' don't match your environment
ERROR: Can't verify hashes for these requirements because we don't have a way to hash version control repositories:
    bigvgan@ git+https://github.com/152334H/BigVGAN.git@HEAD from git+https://github.com/152334H/BigVGAN.git@HEAD (from -r requirements.txt (line 38))
Obtaining file:///content/tortoise-tts-fast
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/152334H/BigVGAN.git to /tmp/pip-install-hi

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip3 install transformers==4.29.2
!pip3 uninstall voicefixer
!pip3 install voicefixer==0.1.2
%cd tortoise-tts-fast
!pwd
!pip3 install git+https://github.com/152334H/BigVGAN.git

  Using cached transformers-4.29.2-py3-none-any.whl.metadata (112 kB)
Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.3
    Uninstalling transformers-4.33.3:
      Successfully uninstalled transformers-4.33.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires librosa>=0.10.0, but you have librosa 0.8.1 which is incompatible.
tts 0.22.0 requires numba>=0.57.0; python_version >= "3.9", but you have numba 0.56.4 which is incompatible.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 1.23.5 which is incompatible.
tts 0.22.0 requires torch>=2.1, but you have torch 1.13.1 which is incompatible.
tts 0.22.0 requires transformers>=4.33.0, but you have transformers 4.29.2 which is incompatible.
Found existing installation: voi

In [ ]:
!pip install pip==24.0
!pip install voicefixer==0.1.2

  Using cached voicefixer-0.1.2-py3-none-any.whl.metadata (8.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.4 MB/s eta 0:00:00
DEPRECATION: voicefixer 0.1.2 has a non-standard dependency specifier streamlit>=1.12.0pyyaml. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of voicefixer or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: torchlibrosa
    Found existing installation: torchlibrosa 0.1.0
    Uninstalling torchlibrosa-0.1.0:
      Successfully uninstalled torchlibrosa-0.1.0


In [ ]:
!git clone https://github.com/coqui-ai/TTS

Cloning into 'TTS'...
remote: Enumerating objects: 32844, done.
remote: Counting objects: 100% (3521/3521), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 32844 (delta 3433), reused 3395 (delta 3395), pack-reused 29323 (from 1)
Receiving objects: 100% (32844/32844), 166.12 MiB | 24.20 MiB/s, done.
Resolving deltas: 100% (23858/23858), done.


In [ ]:
gpt_path = '/content/DL-Art-School/experiments/Learn_OAI_Whisper_20240411_JRB/models/rlg_auto.pth'
text = "Benny, bring me everyone. EVERYONE!!!"

In [ ]:
%cd /content/tortoise-tts-fast/scripts

/content/tortoise-tts-fast/scripts


In [ ]:
!python tortoise_tts.py --preset fast --ar_checkpoint $gpt_path -o "152.wav" $text

Weights downloaded in: /root/.cache/voicefixer/synthesis_module/44100/model.ckpt-1490000_trimed.pt Size: 135613039
Weights downloaded in: /root/.cache/voicefixer/analysis_module/checkpoints/vf.ckpt Size: 489307071
Loading tts...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
config.json: 100% 2.11k/2.11k [00:00<00:00, 16.9MB/s]
pytorch_model.bin: 100% 1.26G/1.26G [01:05<00:00, 19.3MB/s]
preprocessor_config.json: 100% 159/159 [00:00<00:00, 1.09MB/s]
vocab.json: 100% 1.61k/1.61k [00:00<00:00, 11.4MB/s]
tokenizer_config.json: 100% 181/181 [00:00<00:00, 1.44MB/s]
special_tokens_map.json: 100% 85.0/85.0 [00:00<00:00, 741kB/s]
100% ||
Done.
100% ||
Done.
Downloading...
From (original): https://drive.google.com/uc?id=1_cKskUDuvxQJUEBwdgjAxKuDTUW6kPdY
From (red

In [ ]:
import IPython
IPython.display.Audio('/content/tortoise-tts-fast/scripts/results/random_00_00.wav')